# The Hired Hand

**Apprentissage Automatique pour la Prédiction de Placement Professionnel**

[![Ouvrir dans Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Angry-Jay/ML_TheHiredHand/blob/main/ml-the-hired-hand.ipynb)

---

## Table des Matières

1. [Description du Projet et du Jeu de Données](#1-description-du-projet-et-du-jeu-de-données)
   - [1.1 Objectif du Projet](#11-objectif-du-projet)
   - [1.2 Solutions Existantes](#12-solutions-existantes)
   - [1.3 Informations sur le Jeu de Données](#13-informations-sur-le-jeu-de-données)
2. [Importation des Bibliothèques](#2-importation-des-bibliothèques)
3. [Accès aux Données](#3-accès-aux-données)
4. [Analyse Exploratoire du Jeu de Données](#4-analyse-exploratoire-du-jeu-de-données)
   - [4.1 Analyse des Métadonnées](#41-analyse-des-métadonnées)
   - [4.2 Analyse des Valeurs Manquantes](#42-analyse-des-valeurs-manquantes)
   - [4.3 Distributions des Caractéristiques, Mise à l'Échelle et Valeurs Aberrantes](#43-distributions-des-caractéristiques-mise-à-léchelle-et-valeurs-aberrantes)
   - [4.4 Étude de la Variable Cible](#44-étude-de-la-variable-cible)
   - [4.5 Corrélation et Sélection des Caractéristiques](#45-corrélation-et-sélection-des-caractéristiques)
   - [4.6 Clustering Non Supervisé](#46-clustering-non-supervisé)
   - [4.7 Interprétations et Conclusions](#47-interprétations-et-conclusions)
5. [Modèles de Référence et Ensembles ML](#5-modèles-de-référence-et-ensembles-ml)
   - [5.1 Divisions Entraînement/Validation/Test](#51-divisions-entraînementvalidationtest)
   - [5.2 Pipelines et Modèles](#52-pipelines-et-modèles)
   - [5.3 Entraînement et Validation](#53-entraînement-et-validation)
   - [5.4 Test](#54-test)
   - [5.5 Interprétation et Discussion des Résultats](#55-interprétation-et-discussion-des-résultats)
6. [Modèles Améliorés et Optimisation des Hyperparamètres](#6-modèles-améliorés-et-optimisation-des-hyperparamètres)
   - [6.1 Justification des Choix](#61-justification-des-choix)
   - [6.2 Optimisation des Hyperparamètres](#62-optimisation-des-hyperparamètres)
   - [6.3 Résultats Finaux et Analyse](#63-résultats-finaux-et-analyse)
7. [Conclusion et Travaux Futurs](#7-conclusion-et-travaux-futurs)

---

## 1. Description du Projet et du Jeu de Données

### 1.1 Objectif du Projet

Ce projet applique des techniques d'Apprentissage Automatique pour prédire les résultats d'emploi des étudiants diplômés en utilisant le **Jeu de Données de Placement Professionnel**. 

**Objectifs Principaux :**
- **Prédire les résultats d'emploi** (Placé vs Non Placé) en fonction des attributs démographiques, académiques et professionnels
- **Démontrer une méthodologie ML cohérente** de la découverte des données à l'optimisation du modèle
- **Appliquer une analyse de données complète** incluant :
  - Nettoyage et prétraitement des données
  - Analyse Exploratoire des Données (EDA)
  - Ingénierie et sélection des caractéristiques
  - Analyse de corrélation et de clustering
- **Construire et évaluer plusieurs modèles de classification** avec des techniques de validation appropriées
- **Identifier les facteurs clés d'employabilité** à travers l'analyse de l'importance des caractéristiques et l'interprétation du modèle
- **Appliquer les meilleures pratiques ML** incluant les divisions appropriées entraînement/validation/test, la construction de pipelines et l'optimisation des hyperparamètres

---

### 1.2 Solutions Existantes

**Approche Traditionnelle :**

Historiquement, les départements RH et les institutions éducatives s'appuient sur des processus de sélection manuelle avec des filtres heuristiques (par exemple, seuils de moyenne, spécialisations de diplôme spécifiques, seuils d'expérience professionnelle). Cette approche traditionnelle présente plusieurs limitations :
- Chronophage et difficile à mettre à l'échelle
- Subjective et sujette aux biais humains
- Souvent inexacte pour prédire le succès réel du placement professionnel
- Ne parvient pas à capturer les interactions complexes entre plusieurs facteurs

**Solutions d'Apprentissage Automatique :**

Plusieurs approches basées sur le ML existent sur des plateformes comme Kaggle et GitHub pour la prédiction de placement :

**Algorithmes Couramment Utilisés :**
- **Modèles de Référence :** Régression Logistique, K-Plus Proches Voisins (KNN)
- **Modèles Basés sur les Arbres :** Arbres de Décision, Forêt Aléatoire, ExtraTrees
- **Méthodes de Boosting :** XGBoost, AdaBoost, Gradient Boosting
- **Machines à Vecteurs de Support :** SVC avec divers noyaux

**Principales Découvertes de la Littérature :**
- Les méthodes d'ensemble basées sur les arbres (Forêt Aléatoire, XGBoost) surpassent généralement les références plus simples
- Les modèles non linéaires capturent mieux les interactions entre caractéristiques (par exemple, effet combiné de la moyenne et de l'expérience professionnelle)
- L'ingénierie des caractéristiques impacte significativement la performance du modèle
- La gestion appropriée du déséquilibre des classes est cruciale pour des prédictions précises

**Méthodologie Typique :**
1. Analyse Exploratoire des Données (distributions, corrélations, déséquilibre des classes)
2. Pipelines de prétraitement (encodage des variables catégorielles, mise à l'échelle, imputation)
3. Comparaison de modèles utilisant plusieurs métriques : Précision, Exactitude, Rappel, Score-F1, ROC-AUC
4. Optimisation des hyperparamètres utilisant GridSearchCV ou RandomizedSearchCV
5. Analyse de l'importance des caractéristiques pour l'interprétabilité

---

### 1.3 Informations sur le Jeu de Données

**Nom du Jeu de Données :** Jeu de Données de Placement Professionnel

**Source Originale :** [Kaggle - Job Placement Dataset](https://www.kaggle.com/datasets/ahsan81/job-placement-dataset/data)

**Caractéristiques du Jeu de Données :**
- **Type :** Données tabulaires structurées et denses
- **Taille :** Petite à moyenne (215 instances, 13 caractéristiques)
- **Caractéristiques :** Mélange de variables numériques et catégorielles
- **Variable Cible :** Classification binaire (Placé / Non Placé)
- **Qualité :** Propre sans valeurs manquantes ni doublons

**Accès au Jeu de Données :**
- **Dépôt GitHub :** `https://github.com/Angry-Jay/ML_TheHiredHand`
- **URL des Données Brutes :** `https://raw.githubusercontent.com/Angry-Jay/ML_TheHiredHand/main/aug_train.csv` `https://raw.githubusercontent.com/Angry-Jay/ML_TheHiredHand/main/aug_test.csv`

**Aperçu des Caractéristiques :**
- Démographie des étudiants (genre)
- Performance académique (% SSC, % HSC, % Diplôme, % MBA)
- Parcours éducatif (conseil SSC, conseil HSC, spécialisation HSC, type de diplôme, spécialisation MBA)
- Expérience professionnelle
- Scores aux tests d'emploi

## 2. Importation des Bibliothèques

In [ ]:
# Configuration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Prétraitement et Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Sélection et Optimisation de Modèles
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    train_test_split,
)

# Modèles
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Métriques
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)

# Configuration
%matplotlib inline

## 3. Accès aux Données

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/Angry-Jay/ML_TheHiredHand/refs/heads/main/aug_train.csv"

try:
    df = pd.read_csv(DATA_URL)
    
    print("Jeu de données chargé avec succès !")
    print(f"Dimensions : {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    display(df.head())
    
except Exception as e:
    print(f"Erreur lors du chargement des données depuis {DATA_URL}")
    print(f"Détails de l'erreur : {e}")

## 4. Analyse Exploratoire du Jeu de Données

### 4.1 Analyse des Métadonnées

Dans cette section, nous analysons les métadonnées du jeu de données pour comprendre sa structure, ses types de données, sa qualité et ses caractéristiques. Cette exploration initiale aide à identifier :

- **Les dimensions du jeu de données** et son échelle
- **Les types de données des caractéristiques** (numériques vs catégorielles)
- **Les problèmes de qualité des données** (doublons, valeurs manquantes, colonnes non pertinentes)
- **Les propriétés statistiques** des caractéristiques numériques
- **Les préoccupations potentielles de fuite de données**

In [ ]:
# Afficher les informations sur le jeu de données
df.info()

In [ ]:
print("=" * 60)
print("ANALYSE DES DOUBLONS")
print("=" * 60)
duplicates = df.duplicated().sum()
print(f"Nombre de lignes dupliquées : {duplicates}")
if duplicates > 0:
    print("\nLignes dupliquées :")
    display(df[df.duplicated(keep=False)])
else:
    print("Aucune ligne dupliquée trouvée.")


In [ ]:
print("=" * 60)
print("SÉPARATION DES TYPES DE CARACTÉRISTIQUES")
print("=" * 60)

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nCaractéristiques numériques ({len(numerical_cols)}) :")
print(numerical_cols)

print(f"\nCaractéristiques catégorielles ({len(categorical_cols)}) :")
print(categorical_cols)


In [ ]:
print("=" * 60)
print("CARACTÉRISTIQUES NUMÉRIQUES - RÉSUMÉ STATISTIQUE")
print("=" * 60)
display(df[numerical_cols].describe())

In [ ]:
print("=" * 60)
print("CARACTÉRISTIQUES CATÉGORIELLES - VALEURS UNIQUES")
print("=" * 60)

for col in categorical_cols:
    print(f"\n{col} :")
    print(f"  Valeurs uniques : {df[col].nunique()}")
    print(f"  Valeurs : {df[col].unique().tolist()}")

In [ ]:
# Évaluation de la Fuite de Données et Identification de la Variable Cible
print("=" * 60)
print("VARIABLE CIBLE ET ÉVALUATION DE LA FUITE DE DONNÉES")
print("=" * 60)

# Identifier la variable cible
target_col = 'target'
print(f"\nVariable cible : '{target_col}'")
print(f"Classes : {df[target_col].unique().tolist()}")
print(f"\nDistribution des classes :")
print(df[target_col].value_counts())
print(f"\nProportions des classes :")
print(df[target_col].value_counts(normalize=True).round(3))

# Vérifier la composition des caractéristiques
print(f"\n--- Inventaire des Caractéristiques ---")
print(f"Caractéristiques totales : {len(df.columns)}")
print(f"  - Prédicteurs : {len(df.columns) - 1}")
print(f"  - Cible : 1 ('{target_col}')")

# Vérifier les caractéristiques post-placement qui pourraient divulguer des informations
print(f"\n--- Vérification de la Fuite de Données ---")
suspicious_keywords = ['salary', 'offer', 'package', 'compensation', 'hired', 'salaire', 'offre', 'rémunération', 'embauché']
leakage_found = False

for keyword in suspicious_keywords:
    if any(keyword in col.lower() for col in df.columns):
        print(f"AVERTISSEMENT : Caractéristique potentielle de fuite contenant '{keyword}' détectée")
        leakage_found = True

if not leakage_found:
    print("Aucune caractéristique évidente de fuite de données détectée.")
    print("Toutes les caractéristiques représentent des informations disponibles au moment de la prédiction.")

#### Résumé

L'analyse initiale des métadonnées révèle un **jeu de données substantiellement plus grand** par rapport aux études de placement typiques, avec **19 158 instances** sur **14 caractéristiques** (13 prédicteurs et 1 cible). Le jeu de données ne présente **aucun enregistrement dupliqué**, garantissant l'intégrité des données. Cependant, **des valeurs manquantes sont présentes** dans plusieurs caractéristiques, avec les lacunes les plus importantes dans `company_type` (6 140 manquantes, 32,1%), `company_size` (5 938 manquantes, 31,0%), `major_discipline` (2 813 manquantes, 14,7%) et `gender` (4 508 manquantes, 23,5%). Cela nécessite des stratégies d'imputation prudentes ou une gestion des valeurs manquantes lors du prétraitement.

La composition des caractéristiques se compose de **2 prédicteurs numériques** (`city_development_index` et `training_hours`) et **10 prédicteurs catégoriels** représentant la démographie, l'éducation et l'historique d'emploi. De plus, `enrollee_id` sert d'identifiant unique et doit être exclu de la modélisation, tandis que `target` est la variable de résultat binaire. Les caractéristiques catégorielles présentent une **cardinalité variable** : faible cardinalité pour les caractéristiques binaires comme `relevent_experience` (2 valeurs) et `gender` (3 valeurs incluant manquant), cardinalité modérée pour les caractéristiques comme `education_level` (5 niveaux) et `major_discipline` (6 disciplines), et **cardinalité élevée** pour `city` (123 villes uniques) et `experience` (22 niveaux), qui peuvent nécessiter des techniques d'encodage spécialisées telles que l'encodage cible ou l'encodage de fréquence.

La variable cible présente un **déséquilibre de classes significatif**, avec **75,1% des candidats ne cherchant pas de changement d'emploi** (classe 0) et seulement **24,9% recherchant activement un changement** (classe 1), donnant un ratio de déséquilibre de **3,01:1**. Ce déséquilibre substantiel doit être traité pendant l'entraînement du modèle grâce à des techniques telles que la pondération des classes, le rééchantillonnage (SMOTE/sous-échantillonnage) ou l'utilisation de métriques d'évaluation robustes au déséquilibre (score F1, ROC-AUC, courbes de précision-rappel). **Aucune préoccupation de fuite de données** n'a été identifiée ; toutes les caractéristiques représentent des informations collectées lors de l'inscription à la formation, garantissant la validité du modèle pour prédire les intentions réelles de changement d'emploi.

### 4.2 Analyse des Valeurs Manquantes

In [ ]:
print("=" * 60)
print("ANALYSE DES VALEURS MANQUANTES")
print("=" * 60)

# Nombre de valeurs manquantes
missing_counts = df.isnull().sum()
print("\nValeurs manquantes par caractéristique :")
print(missing_counts)

# Pourcentage de valeurs manquantes
print("\n" + "=" * 60)
print("POURCENTAGE DE VALEURS MANQUANTES")
print("=" * 60)
missing_percentages = (df.isnull().sum() / len(df) * 100).round(2)
print(missing_percentages)

# Statistiques récapitulatives
total_missing = df.isnull().sum().sum()
total_cells = df.shape[0] * df.shape[1]
print("\n" + "=" * 60)
print("RÉSUMÉ")
print("=" * 60)
print(f"Total de valeurs manquantes : {total_missing}")
print(f"Total de cellules : {total_cells}")
print(f"Taux de valeurs manquantes global : {(total_missing / total_cells * 100):.2f}%")
print(f"Caractéristiques avec valeurs manquantes : {(missing_counts > 0).sum()} sur {len(df.columns)}")
print(f"Caractéristiques complètes : {(missing_counts == 0).sum()} sur {len(df.columns)}")

In [ ]:
# Visualiser les valeurs manquantes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Diagramme en barres des valeurs manquantes
missing_data = df.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

if len(missing_data) > 0:
    axes[0].bar(range(len(missing_data)), missing_data.values, edgecolor='black', alpha=0.7, color='orange')
    axes[0].set_xticks(range(len(missing_data)))
    axes[0].set_xticklabels(missing_data.index, rotation=45, ha='right')
    axes[0].set_title('Nombre de Valeurs Manquantes par Caractéristique', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Caractéristiques')
    axes[0].set_ylabel('Nombre de Valeurs Manquantes')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Ajouter les étiquettes de comptage
    for i, v in enumerate(missing_data.values):
        axes[0].text(i, v + 100, str(v), ha='center', va='bottom')
else:
    axes[0].text(0.5, 0.5, 'Aucune Valeur Manquante', ha='center', va='center', fontsize=14)
    axes[0].set_xlim(0, 1)
    axes[0].set_ylim(0, 1)

# Graphique de pourcentage
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)
missing_pct = missing_pct[missing_pct > 0].sort_values(ascending=False)

if len(missing_pct) > 0:
    axes[1].bar(range(len(missing_pct)), missing_pct.values, edgecolor='black', alpha=0.7, color='red')
    axes[1].set_xticks(range(len(missing_pct)))
    axes[1].set_xticklabels(missing_pct.index, rotation=45, ha='right')
    axes[1].set_title('Pourcentage de Valeurs Manquantes par Caractéristique', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Caractéristiques')
    axes[1].set_ylabel('Pourcentage (%)')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Ajouter les étiquettes de pourcentage
    for i, v in enumerate(missing_pct.values):
        axes[1].text(i, v + 0.5, f'{v}%', ha='center', va='bottom')
else:
    axes[1].text(0.5, 0.5, 'Aucune Valeur Manquante', ha='center', va='center', fontsize=14)
    axes[1].set_xlim(0, 1)
    axes[1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Analyse des modèles de valeurs manquantes
print("=" * 60)
print("MODÈLES DE VALEURS MANQUANTES")
print("=" * 60)

# Caractéristiques avec valeurs manquantes
features_with_missing = df.columns[df.isnull().any()].tolist()

if len(features_with_missing) > 0:
    print(f"\nCaractéristiques avec valeurs manquantes ({len(features_with_missing)}) : {features_with_missing}")
    
    # Vérifier la co-occurrence des valeurs manquantes
    print("\n" + "=" * 60)
    print("CO-OCCURRENCE DES VALEURS MANQUANTES")
    print("=" * 60)
    
    # Vérifier les combinaisons courantes
    if 'company_size' in features_with_missing and 'company_type' in features_with_missing:
        both_missing = df['company_size'].isnull() & df['company_type'].isnull()
        print(f"\ncompany_size ET company_type toutes deux manquantes : {both_missing.sum()} ({both_missing.sum() / len(df) * 100:.2f}%)")
    
    if 'company_size' in features_with_missing and 'company_type' in features_with_missing and 'experience' in features_with_missing:
        all_three = df['company_size'].isnull() & df['company_type'].isnull() & df['experience'].isnull()
        print(f"company_size ET company_type ET experience toutes manquantes : {all_three.sum()} ({all_three.sum() / len(df) * 100:.2f}%)")
    
    # Distribution du nombre de valeurs manquantes par ligne
    print("\n" + "=" * 60)
    print("DISTRIBUTION DES VALEURS MANQUANTES PAR LIGNE")
    print("=" * 60)
    missing_per_row = df.isnull().sum(axis=1)
    print(f"\nDistribution des valeurs manquantes :")
    print(missing_per_row.value_counts().sort_index())
    
    # Lignes avec au moins une valeur manquante
    rows_with_missing = df.isnull().any(axis=1).sum()
    print(f"\n--- Résumé ---")
    print(f"Total de lignes avec au moins une valeur manquante : {rows_with_missing} ({rows_with_missing / len(df) * 100:.2f}%)")
    
    # Lignes avec toutes les valeurs complètes
    complete_rows = (~df.isnull().any(axis=1)).sum()
    print(f"Lignes complètes (aucune valeur manquante) : {complete_rows} ({complete_rows / len(df) * 100:.2f}%)")
    
    # Modèles de valeurs manquantes les plus courants
    print("\n" + "=" * 60)
    print("TOP 5 DES MODÈLES DE VALEURS MANQUANTES")
    print("=" * 60)
    missing_patterns = df[features_with_missing].isnull().astype(int)
    pattern_counts = missing_patterns.groupby(features_with_missing).size().sort_values(ascending=False).head(5)
    
    for idx, (pattern, count) in enumerate(pattern_counts.items(), 1):
        missing_features = [feat for feat, is_missing in zip(features_with_missing, pattern) if is_missing == 1]
        if missing_features:
            print(f"\n{idx}. Manquants : {missing_features}")
            print(f"   Nombre : {count} ({count / len(df) * 100:.2f}%)")
        else:
            print(f"\n{idx}. Aucune valeur manquante")
            print(f"   Nombre : {count} ({count / len(df) * 100:.2f}%)")
            
else:
    print("\nAucune valeur manquante détectée dans le jeu de données.")

#### Résumé

L'analyse des valeurs manquantes révèle une **incomplétude significative des données** affectant **53,26% de toutes les lignes** (10 203 instances), tandis que seulement **46,74%** (8 955 instances) sont complètes. Sur **14 caractéristiques**, **8 contiennent des valeurs manquantes** avec un **modèle hiérarchique** : **company_type** (6 140 manquantes, 32,05%) et **company_size** (5 938 manquantes, 30,99%) dominent, suivies de **gender** (4 508 manquantes, 23,53%) et **major_discipline** (2 813 manquantes, 14,68%). Des valeurs manquantes de niveau inférieur apparaissent dans **education_level** (460 manquantes, 2,40%), **last_new_job** (423 manquantes, 2,21%), **enrolled_university** (386 manquantes, 2,01%) et **experience** (65 manquantes, 0,34%). Le taux global de valeurs manquantes du jeu de données est de **7,73%** du total des cellules.

**L'analyse des modèles** révèle une **co-occurrence systématique** des valeurs manquantes, confirmant un comportement non aléatoire (MNAR). Les principaux modèles de valeurs manquantes sont : **(1) Lignes complètes sans valeurs manquantes : 46,74%** ; **(2) company_size ET company_type toutes deux manquantes : 14,50%** (2 777 lignes) — indiquant fortement des candidats sans emploi ou des étudiants ; **(3) Seulement gender manquant : 11,61%** (2 224 lignes) ; **(4) major_discipline, company_size ET company_type manquants : 4,42%** (847 lignes) ; **(5) gender, company_size ET company_type manquants : 4,36%** (835 lignes). Notamment, **5 360 lignes (27,98%)** ont les deux caractéristiques d'emploi manquantes ensemble, tandis que seulement **20 lignes (0,10%)** ont les trois caractéristiques liées à l'emploi (company_size, company_type, experience) manquantes simultanément. La distribution des valeurs manquantes par ligne montre que la plupart des lignes affectées ont **1-3 caractéristiques manquantes**, avec une fréquence décroissante pour les nombres plus élevés (628 lignes avec 4 manquantes, 176 avec 5, 62 avec 6 et seulement 12 avec 7).

**Stratégie de prétraitement :** Pour **company_size et company_type**, nous allons **créer une catégorie explicite « Non Employé »** plutôt que d'imputer des valeurs, car le modèle de co-occurrence de 27,98% représente clairement des candidats sans emploi actuel (étudiants/débutants) où ces champs ne sont véritablement pas applicables — l'imputation introduirait de fausses informations et obscurcirait cet indicateur significatif du statut d'emploi. Pour les **caractéristiques à faible taux de valeurs manquantes** (education_level, enrolled_university, last_new_job, experience toutes <3%), nous appliquerons une **imputation par mode** car leurs valeurs manquantes sporadiques suggèrent des lacunes aléatoires de collecte de données plutôt que des modèles systématiques, et leur faible prévalence minimise l'impact sur la validité du modèle. Pour **gender et major_discipline**, nous allons **créer des catégories « Inconnu »** car leurs valeurs manquantes indépendantes substantielles (11,61% pour gender seul, 14,68% pour major_discipline) indiquent une réticence à fournir des données ou des préoccupations de confidentialité plutôt qu'une non-applicabilité, et préserver ce signal « non fourni » peut lui-même être prédictif du comportement de changement d'emploi. De plus, nous allons **créer des indicateurs binaires de valeurs manquantes** (`has_employment_info`, `gender_provided`, `education_complete`) car les 53,26% de lignes incomplètes peuvent présenter des comportements de recherche d'emploi distincts, et ces indicateurs pourraient capturer des modèles précieux pour la prédiction.

### 4.3 Distributions des Caractéristiques, Mise à l'Échelle et Valeurs Aberrantes

In [ ]:
# Visualiser les distributions des caractéristiques numériques (excluant enrollee_id et target)
numerical_features_for_viz = [col for col in numerical_cols if col not in ['enrollee_id', 'target']]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, col in enumerate(numerical_features_for_viz):
    axes[idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[idx].set_title(f'Distribution de {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Fréquence')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Détecter les valeurs aberrantes avec des diagrammes en boîte (excluant enrollee_id et target)
numerical_features_for_viz = [col for col in numerical_cols if col not in ['enrollee_id', 'target']]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for idx, col in enumerate(numerical_features_for_viz):
    axes[idx].boxplot(df[col].dropna(), vert=True)
    axes[idx].set_title(f'{col}', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Valeur')
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Détection quantitative des valeurs aberrantes en utilisant la méthode IQR (excluant enrollee_id et target)
print("=" * 60)
print("DÉTECTION DES VALEURS ABERRANTES (MÉTHODE IQR)")
print("=" * 60)

numerical_features_for_analysis = [col for col in numerical_cols if col not in ['enrollee_id', 'target']]

for col in numerical_features_for_analysis:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    
    print(f"\n{col} :")
    print(f"  Q1 : {Q1:.2f}, Q3 : {Q3:.2f}, IQR : {IQR:.2f}")
    print(f"  Limites : [{lower_bound:.2f}, {upper_bound:.2f}]")
    print(f"  Valeurs aberrantes détectées : {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    
    if len(outliers) > 0:
        print(f"  Valeur aberrante minimale : {df[col][outliers.index].min():.2f}")
        print(f"  Valeur aberrante maximale : {df[col][outliers.index].max():.2f}")

In [ ]:
# Distribution des caractéristiques catégorielles (visualiser les catégories principales pour les caractéristiques à haute cardinalité)
categorical_features = categorical_cols.copy()

fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.ravel()

for idx, col in enumerate(categorical_features[:10]):
    # Pour les caractéristiques à haute cardinalité, afficher seulement les 10 premières
    value_counts = df[col].value_counts()
    
    if len(value_counts) > 10:
        value_counts = value_counts.head(10)
        title_suffix = " (Top 10)"
    else:
        title_suffix = ""
    
    axes[idx].bar(range(len(value_counts)), value_counts.values, edgecolor='black', alpha=0.7)
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right', fontsize=8)
    axes[idx].set_title(f'{col}{title_suffix}', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=9)
    axes[idx].set_ylabel('Nombre', fontsize=9)
    axes[idx].grid(axis='y', alpha=0.3)
    
    # Ajouter les étiquettes de comptage sur les barres
    for i, v in enumerate(value_counts.values):
        axes[idx].text(i, v + max(value_counts.values)*0.01, str(v), ha='center', va='bottom', fontsize=8)

# Supprimer les sous-graphes vides
for i in range(len(categorical_features[:10]), 12):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Imprimer les statistiques récapitulatives pour toutes les caractéristiques catégorielles
print("=" * 60)
print("RÉSUMÉ DE LA CARDINALITÉ DES CARACTÉRISTIQUES CATÉGORIELLES")
print("=" * 60)
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"{col} : {unique_count} valeurs uniques")

#### Résumé

**Distributions des Caractéristiques Numériques :** Le jeu de données ne contient que **deux prédicteurs numériques significatifs**. L'histogramme de `city_development_index` révèle une **distribution fortement asymétrique vers la gauche** avec une concentration massive aux indices de développement les plus élevés (la plage 0,9-0,95 montre ~7 000+ candidats), indiquant que la plupart des participants à la formation proviennent de centres urbains hautement développés. Le diagramme en boîte confirme les valeurs aberrantes minimales autour de 0,448-0,47. En contraste, `training_hours` affiche une **distribution asymétrique vers la droite** avec le mode aux basses barres (0-50 heures montrant ~2 700 candidats), suivi par des fréquences progressivement décroissantes vers les heures d'entraînement plus élevées. Le diagramme en boîte illustre dramatiquement les **valeurs aberrantes supérieures étendues** (984 instances, 5,1%) s'étirant de ~185 à 336 heures, représentant des candidats ayant un engagement d'entraînement exceptionnel bien au-delà de la limite supérieure IQR de 88 heures.

**Analyse des Valeurs Aberrantes :** Nous allons **conserver toutes les valeurs aberrantes** plutôt que de les supprimer. Les valeurs aberrantes de `city_development_index` (17 instances, 0,1%) représentent des candidats de villes moins développées — une minorité légitime et potentiellement informative dont le comportement de recherche d'emploi peut différer de la majorité urbaine. Les valeurs aberrantes de `training_hours` (984 instances, 5,1%) sont particulièrement précieuses : les candidats investissant 185-336 heures dans la formation démontrent un engagement extrême qui pourrait fortement prédire les intentions de changement d'emploi, indiquant soit un perfectionnement actif pour la transition de carrière, soit une formation obligatoire par l'employeur pour les rôles actuels. Supprimer ces 5% de candidats supprimerait une cohorte comportementalement distincte dont le statut de valeur aberrante porte lui-même un signal prédictif. Les valeurs aberrantes représentent une variance comportementale authentique, pas des erreurs de mesure.

**Distributions des Caractéristiques Catégorielles :** Les visualisations révèlent des **déséquilibres prononcés dans toutes les caractéristiques**. La **Ville** (montrant les 10 premières sur 123) se concentre lourdement à city_103 (4 355 candidats), les autres villes ayant une représentation dramatiquement plus faible. Le **Genre** affiche une **dominance masculine forte** (13 221 hommes vs 1 238 femmes vs 191 autres), créant un déséquilibre de 10,7:1. L'**Expérience pertinente** est fortement asymétrique vers « Has relevant experience » (13 792 vs 5 366). L'**Université inscrite** montre la plupart comme « no_enrollment » (13 817 vs 3 757 à temps plein vs 1 198 à temps partiel). Le **Niveau d'éducation** est dominé par les **Diplômés** (11 598) suivis par les Masters (4 361), avec une représentation minimale pour l'école secondaire, le doctorat et l'école primaire. La **Discipline majeure** favorise massivement les **STEM** (14 492), éclipsant les diplômes commerciaux, autres, sciences humaines, arts et pas de spécialité. L'**Expérience** (montrant les 10 premières sur 22 niveaux) montre une distribution large avec « >20 » ans en tête (3 286), mais relativement équilibrée sur les plages 5-20 ans. La **Taille de l'entreprise** culmine à « 50-99 » (3 884) avec des fréquences décroissantes pour les tailles plus importantes. Le **Type d'entreprise** est dominé par « **Pvt Ltd** » (10 817), dépassant largement les startups financées, le secteur public, les jeunes startups, les ONG et autres. Le **Dernier nouvel emploi** affiche « 1 » an comme le plus courant (8 040), déclinant pour les écarts plus longs. Ces déséquilibres nécessiteront des stratégies d'encodage prudentes : encodage cible/fréquence pour les caractéristiques à haute cardinalité (city, experience), encodage one-hot pour les caractéristiques à faible cardinalité et attention au déséquilibre des classes lors de l'entraînement du modèle pour éviter les biais vers les classes majoritaires.

### 4.4 Étude de la Variable Cible

In [ ]:
# Distribution de la variable cible
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Diagramme de comptage
target_counts = df['target'].value_counts()
axes[0].bar(target_counts.index, target_counts.values, edgecolor='black', alpha=0.7, color=['red', 'green'])
axes[0].set_title('Distribution de la Cible (Nombre)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Cible (0=Pas Recherche, 1=Recherche Changement Emploi)')
axes[0].set_ylabel('Nombre')
for i, v in enumerate(target_counts.values):
    axes[0].text(i, v + 100, str(v), ha='center', va='bottom', fontweight='bold')

# Diagramme circulaire
axes[1].pie(target_counts.values, labels=['Pas Recherche (0)', 'Recherche (1)'], autopct='%1.1f%%', 
            startangle=90, colors=['red', 'green'], explode=(0.05, 0))
axes[1].set_title('Distribution de la Cible (Proportion)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("=" * 60)
print("ANALYSE DE LA VARIABLE CIBLE")
print("=" * 60)
print(f"\nDistribution des classes :")
print(target_counts)
print(f"\nProportions des classes :")
print(df['target'].value_counts(normalize=True).round(3))
print(f"\nRatio de déséquilibre des classes : {target_counts.max() / target_counts.min():.2f}:1")

In [ ]:
# Comparaison des caractéristiques numériques par classe cible
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes = axes.ravel()

# Exclure enrollee_id de la comparaison (c'est juste un identifiant)
numerical_features_for_comparison = [col for col in numerical_cols if col not in ['enrollee_id', 'target']]

for idx, col in enumerate(numerical_features_for_comparison):
    looking = df[df['target'] == 1.0][col]
    not_looking = df[df['target'] == 0.0][col]
    
    axes[idx].hist([not_looking, looking], bins=15, label=['Pas Recherche (0)', 'Recherche (1)'], 
                   edgecolor='black', alpha=0.7, color=['red', 'green'])
    axes[idx].set_title(f'{col} par Cible', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Fréquence')
    axes[idx].legend()
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Comparaison statistique des caractéristiques numériques par classe cible
print("=" * 60)
print("CARACTÉRISTIQUES NUMÉRIQUES - COMPARAISON DES MOYENNES PAR CIBLE")
print("=" * 60)

numerical_features_for_comparison = [col for col in numerical_cols if col not in ['enrollee_id', 'target']]
comparison = df.groupby('target')[numerical_features_for_comparison].mean()
comparison.index = ['Pas Recherche (0)', 'Recherche (1)']
print("\nValeurs moyennes par classe cible :")
print(comparison.round(2))

print("\n" + "=" * 60)
print("DIFFÉRENCE (Recherche - Pas Recherche)")
print("=" * 60)
difference = comparison.loc['Recherche (1)'] - comparison.loc['Pas Recherche (0)']
print(difference.round(2))

# Visualiser la comparaison des moyennes
comparison.T.plot(kind='bar', figsize=(10, 5), edgecolor='black', alpha=0.7, color=['red', 'green'])
plt.title('Comparaison des Moyennes des Caractéristiques Numériques par Cible', fontsize=13, fontweight='bold')
plt.xlabel('Caractéristiques')
plt.ylabel('Valeur Moyenne')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Cible')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Caractéristiques catégorielles vs cible - taux de changement d'emploi
categorical_features_for_analysis = [col for col in categorical_cols]

# Limiter aux catégories principales pour les caractéristiques à haute cardinalité comme 'city'
fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.ravel()

for idx, col in enumerate(categorical_features_for_analysis[:10]):
    if col == 'city':
        # Pour city, afficher seulement les 10 premières villes
        top_cities = df[col].value_counts().head(10).index
        df_subset = df[df[col].isin(top_cities)]
        ct = pd.crosstab(df_subset[col], df_subset['target'], normalize='index') * 100
    else:
        ct = pd.crosstab(df[col], df['target'], normalize='index') * 100
    
    ct.plot(kind='bar', ax=axes[idx], edgecolor='black', alpha=0.7, color=['red', 'green'])
    axes[idx].set_title(f'Taux de Changement d\'Emploi par {col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Pourcentage (%)')
    axes[idx].legend(['Pas Recherche (0)', 'Recherche (1)'], fontsize=7)
    axes[idx].tick_params(axis='x', rotation=45, labelsize=8)
    axes[idx].grid(axis='y', alpha=0.3)

# Supprimer les sous-graphes vides
for i in range(len(categorical_features_for_analysis[:10]), 12):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

#### Résumé

**Distribution de la Classe Cible :** La variable cible présente un **déséquilibre de classe sévère** avec **14 381 candidats (75,1%) ne cherchant pas de changement d'emploi** par rapport à **4 777 candidats (24,9%) recherchant activement un changement**, donnant un **ratio de déséquilibre de 3,01:1**. Ce déséquilibre significatif nécessitera un traitement prudent pendant l'entraînement du modèle grâce à la pondération des classes, au rééchantillonnage SMOTE ou à l'échantillonnage stratifié pour éviter que le modèle ne se contente de prédictions de classe majoritaire et ne réalise une précision trompeusement élevée tout en échouant à identifier les candidats cherchant un emploi.

**Caractéristiques Numériques vs Cible :** Les histogrammes comparatifs révèlent un **pouvoir discriminatoire minimal** pour les deux caractéristiques numériques. `city_development_index` montre des distributions presque identiques pour les deux classes, toutes deux concentrées aux indices de développement élevés (plage 0,9-0,95). La comparaison des moyennes confirme cela : **Pas Recherche : 0,83** vs **Recherche : 0,81** — une différence triviale de seulement **-0,02**. De même, `training_hours` affiche des distributions qui se chevauchent avec les deux classes culminant à 0-50 heures, bien que les candidats cherchant un emploi montrent légèrement une diffusion plus large vers les heures d'entraînement plus élevées. La différence moyenne est également minimale : **Pas Recherche : 65,70** vs **Recherche : 64,35** — une différence de **-1,35 heures**. La visualisation du diagramme en barres illustre dramatiquement ces différences négligeables, les deux caractéristiques montrant des valeurs moyennes presque identiques entre les classes. Cela suggère que **ni l'une ni l'autre des caractéristiques numériques ne fournit un signal prédictif fort** seule, indiquant que le modèle devra s'appuyer fortement sur les caractéristiques catégorielles et les interactions entre caractéristiques.

**Caractéristiques Catégorielles vs Cible :** L'analyse des caractéristiques catégorielles révèle une **variation substantielle de la propension à la recherche d'emploi** entre différents segments. **La Ville** montre une variance dramatique, city_21 présentant le **taux de changement d'emploi le plus élevé (~60%)** tandis que la plupart des autres villes principales se situent autour de 10-20%. **Le Genre** révèle que les **femmes (26%) ont des taux de recherche d'emploi plus élevés que les hommes (22%)**, contredisant les hypothèses courantes. **L'Expérience Pertinente** montre un modèle frappant : les candidats **sans expérience pertinente ont un taux de changement d'emploi de 35%** par rapport à seulement **20% pour les candidats expérimentés** — suggérant que les professionnels moins établis sont plus mobiles. **L'Université Inscrite** démontre que les **étudiants à temps plein ont la mobilité la plus élevée (38%)**, suivis par les étudiants à temps partiel (25%), tandis que les non-inscrits affichent les taux les plus bas (20%). Le **Niveau d'Éducation** révèle que les **Diplômés et Diplômés de l'École Secondaire** ont une mobilité plus élevée (~28%) comparés aux Masters, Doctorats et Diplômés de l'École Primaire. La **Discipline Majeure** affiche des taux relativement **équilibrés (20-28%) dans tous les domaines**. Les **Niveaux d'Expérience** affichent une intéressante non-linéarité : les **candidats débutants (<1 an) et très expérimentés (>20 ans) affichent des taux de recherche d'emploi plus élevés (~25-30%)** comparés aux professionnels en milieu de carrière (15-20%). La **Taille de l'Entreprise** révèle que les **petites entreprises (<10, 10/49) ont une attrition plus élevée (~23%)** par rapport aux plus grandes corporations (~15-18%). Le **Type d'Entreprise** affiche que les **Jeunes Startups ont la mobilité la plus élevée (~24%)** tandis que le **Secteur Public** affiche la mobilité la plus faible (~18%). Le **Dernier Nouvel Emploi** démontre que les candidats **ayant « jamais » changé d'emploi ont le taux le plus élevé (~32%)**, suggérant les candidats pour leur premier emploi, tandis que les changeurs récents (1-2 ans) montrent ~22-25%. Ces modèles catégoriels seront **cruciaux pour les prédictions du modèle**, car ils révèlent des segments comportementaux distincts avec une propension variable à la recherche d'emploi que les caractéristiques numériques ne parviennent pas à capturer.

### 4.5 Corrélation et Sélection des Caractéristiques

In [ ]:
# Analyse de corrélation des caractéristiques numériques
print("=" * 60)
print("CORRÉLATION DES CARACTÉRISTIQUES NUMÉRIQUES")
print("=" * 60)

# Corréler seulement les caractéristiques numériques significatives (exclure enrollee_id)
numerical_features_for_correlation = [col for col in numerical_cols if col not in ['enrollee_id']]

# Calculer la matrice de corrélation
corr_matrix = df[numerical_features_for_correlation].corr()

print("\nMatrice de Corrélation :")
print(corr_matrix.round(3))

# Visualiser la carte thermique de corrélation
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, fmt='.3f')
plt.title('Carte Thermique de Corrélation - Caractéristiques Numériques', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

#### Corrélation des Caractéristiques Catégorielles (V de Cramér)

Le V de Cramér mesure l'association entre variables catégorielles (0 = aucune, 1 = parfaite).
- **V < 0,1** : Négligeable | **0,1-0,3** : Faible | **0,3-0,5** : Modérée | **V ≥ 0,5** : Forte

In [ ]:
from scipy.stats.contingency import association

# Calculer la matrice V de Cramér pour les caractéristiques catégorielles
categorical_features_for_corr = [col for col in categorical_cols]
n_features = len(categorical_features_for_corr)
cramers_matrix = np.zeros((n_features, n_features))

for i, col1 in enumerate(categorical_features_for_corr):
    for j, col2 in enumerate(categorical_features_for_corr):
        if i == j:
            cramers_matrix[i, j] = 1.0
        elif i < j:
            mask = df[col1].notna() & df[col2].notna()
            contingency = pd.crosstab(df.loc[mask, col1], df.loc[mask, col2])
            v = association(contingency, method='cramer')
            cramers_matrix[i, j] = v
            cramers_matrix[j, i] = v

# Visualiser
cramers_df = pd.DataFrame(cramers_matrix, index=categorical_features_for_corr, columns=categorical_features_for_corr)

plt.figure(figsize=(10, 8))
sns.heatmap(cramers_df, annot=True, cmap='YlOrRd', square=True, fmt='.2f', vmin=0, vmax=1)
plt.title("V de Cramér - Corrélation des Caractéristiques Catégorielles", fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

#### Tests du Chi-Carré : Caractéristiques Catégorielles vs Cible

Test d'indépendance pour évaluer l'association entre chaque caractéristique catégorielle et la variable cible.

In [ ]:
from scipy.stats import chi2_contingency

# Tests du chi-carré : caractéristiques catégorielles vs cible
chi2_results = []

for col in categorical_cols:
    mask = df[col].notna()
    contingency = pd.crosstab(df.loc[mask, col], df.loc[mask, 'target'])
    chi2, p_value, dof, _ = chi2_contingency(contingency)
    v = association(contingency, method='cramer')
    
    chi2_results.append({
        'Caractéristique': col,
        'Chi-carré': round(chi2, 2),
        'p-valeur': p_value,
        "V de Cramér": round(v, 3)
    })

chi2_df = pd.DataFrame(chi2_results).sort_values("V de Cramér", ascending=False)
print("Tests du Chi-Carré : Caractéristiques Catégorielles vs Cible")
print(chi2_df.to_string(index=False))

# Visualiser
plt.figure(figsize=(8, 5))
plt.barh(chi2_df['Caractéristique'], chi2_df["V de Cramér"], color='steelblue', edgecolor='black')
plt.xlabel("V de Cramér")
plt.title("Force d'Association avec la Cible", fontsize=12, fontweight='bold')
plt.axvline(x=0.1, color='orange', linestyle='--', label='Faible (0.1)')
plt.axvline(x=0.3, color='red', linestyle='--', label='Modérée (0.3)')
plt.legend()
plt.tight_layout()
plt.show()

#### Résumé

**Corrélation numérique :** Les deux caractéristiques numériques (`city_development_index` et `training_hours`) sont indépendantes (r = 0,002). Seul `city_development_index` montre une corrélation modérée avec la cible (r = -0,34), indiquant que les candidats des villes moins développées cherchent davantage à changer d'emploi.

**Corrélation catégorielle (V de Cramér) :** La matrice révèle des associations modérées entre certaines caractéristiques liées à l'expérience professionnelle :
- `relevent_experience` ↔ `experience` (V = 0,40)
- `relevent_experience` ↔ `enrolled_university` / `last_new_job` (V = 0,39)
- `relevent_experience` ↔ `education_level` (V = 0,32)

Ces corrélations sont logiques (progression de carrière) mais restent modérées, sans multicolinéarité sévère.

**Association avec la cible (Chi-carré) :** Toutes les caractéristiques catégorielles sont statistiquement significatives (p < 0,05). Classement par force d'association :
- **Modérée (V ≥ 0,3) :** `city` (0,396) — meilleur prédicteur catégoriel
- **Faible (0,1-0,3) :** `experience` (0,192), `enrolled_university` (0,156), `relevent_experience` (0,128)
- **Négligeable (V < 0,1) :** `education_level`, `last_new_job`, `company_size`, `company_type`, `major_discipline`, `gender`

**Conclusion :** `city` et `city_development_index` sont les prédicteurs les plus forts. Les caractéristiques liées à l'expérience (`experience`, `enrolled_university`, `relevent_experience`) forment un second groupe de prédicteurs utiles. Les caractéristiques démographiques (`gender`, `major_discipline`) ont un pouvoir prédictif faible mais restent significatives.

### 4.6 Clustering Non Supervisé

In [ ]:
# Préparation des données pour le clustering
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Sélection des features (sans enrollee_id et target)
X_cluster = df.drop(['enrollee_id', 'target'], axis=1)

# Identifier les colonnes
num_cols = X_cluster.select_dtypes(include=np.number).columns.tolist()
cat_cols = X_cluster.select_dtypes(include='object').columns.tolist()

# Pipeline de prétraitement
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]), cat_cols)
])

# Transformer les données
X_processed = preprocessor.fit_transform(X_cluster)
print(f"Données prétraitées: {X_processed.shape[0]} samples, {X_processed.shape[1]} features")

In [ ]:
# Méthode du coude (Elbow Method)
from sklearn.cluster import KMeans

inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_processed)
    inertias.append(kmeans.inertia_)

# Visualisation
plt.figure(figsize=(8, 5))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie')
plt.title('Méthode du Coude', fontsize=13, fontweight='bold')
plt.xticks(K_range)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Application K-Means (choisir k après avoir vu le coude)
k_optimal = 4  # À ajuster selon le graphique du coude

kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_processed)

# Réduction PCA pour visualisation
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_processed)

# Visualisation des clusters
plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.5, s=10)
plt.colorbar(scatter, label='Cluster')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title(f'K-Means Clustering (k={k_optimal}) - Projection PCA', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Variance expliquée par PCA: {pca.explained_variance_ratio_.sum()*100:.1f}%")

In [ ]:
# Analyse des clusters vs target
df_clusters = df.copy()
df_clusters['cluster'] = clusters

# Distribution du target par cluster
cluster_analysis = df_clusters.groupby('cluster')['target'].agg(['count', 'sum', 'mean'])
cluster_analysis.columns = ['Total', 'Looking for Change', 'Rate (%)']
cluster_analysis['Rate (%)'] = (cluster_analysis['Rate (%)'] * 100).round(1)
print("Distribution du target par cluster:")
print(cluster_analysis)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Taille des clusters
axes[0].bar(cluster_analysis.index, cluster_analysis['Total'], color='steelblue', edgecolor='black')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Nombre de candidats')
axes[0].set_title('Taille des clusters', fontweight='bold')

# Taux de recherche d'emploi par cluster
colors = plt.cm.RdYlGn_r(cluster_analysis['Rate (%)'] / 100)
axes[1].bar(cluster_analysis.index, cluster_analysis['Rate (%)'], color=colors, edgecolor='black')
axes[1].axhline(y=df['target'].mean()*100, color='red', linestyle='--', label=f'Moyenne globale ({df["target"].mean()*100:.1f}%)')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Taux de recherche (%)')
axes[1].set_title('Taux de recherche d\'emploi par cluster', fontweight='bold')
axes[1].legend()

plt.tight_layout()
plt.show()

#### Résumé

Le clustering K-Means (k=4) révèle **4 segments de candidats** avec des comportements distincts vis-à-vis de la recherche d'emploi:

- **Cluster 1** (~8,800 candidats, 13%): Profil le plus stable, taux de recherche bien inférieur à la moyenne
- **Cluster 3** (~4,800 candidats, 47%): Profil à **haut risque d'attrition**, taux presque 2x supérieur à la moyenne globale (24.9%)
- **Clusters 0 et 2**: Comportements intermédiaires (18% et 27%)

La variance expliquée par PCA (29.6%) est limitée en raison du grand nombre de features catégorielles encodées, mais les clusters restent visuellement distincts. Le fait que le clustering non-supervisé identifie des groupes corrélés au target **sans l'avoir utilisé** confirme que les features contiennent des patterns prédictifs exploitables.

### 4.7 Interprétations et Conclusions

L'analyse exploratoire a révélé les points clés suivants :

**Données :**
- 19 158 candidats, 12 caractéristiques prédictives (2 numériques, 10 catégorielles)
- Déséquilibre de classes : 75% / 25% (ratio 3:1)
- Valeurs manquantes significatives (53% des lignes affectées), notamment `company_size` et `company_type` (candidats sans emploi)

**Caractéristiques les plus prédictives :**
| Caractéristique | Type | Association avec cible |
|---------|------|------------------------|
| city | Catégorielle | V = 0,396 (modérée) |
| city_development_index | Numérique | r = -0,342 (modérée) |
| experience | Catégorielle | V = 0,192 (faible) |
| enrolled_university | Catégorielle | V = 0,156 (faible) |
| relevent_experience | Catégorielle | V = 0,128 (faible) |

**Perspicacités Métier :**
- Les candidats des **villes moins développées** cherchent davantage à changer d'emploi
- Les **étudiants à temps plein** et candidats **sans expérience pertinente** sont plus mobiles
- Le clustering identifie un segment à **haut risque (47%)** vs un segment **stable (13%)**

**Implications pour la Modélisation :**
1. Gérer le déséquilibre de classes (class_weight, SMOTE)
2. Traiter les valeurs manquantes (imputation + catégorie « Inconnu »)
3. Encoder les caractéristiques à haute cardinalité (`city` : 123 valeurs)
4. Privilégier les modèles non-linéaires (Random Forest, XGBoost) pour capturer les interactions

---

## 5. Modèles de Référence et Ensembles ML

### 5.1 Divisions Entraînement/Validation/Test

Division des données en **3 ensembles distincts** pour une évaluation rigoureuse :

| Ensemble | Proportion | Rôle |
|----------|------------|------|
| **Entraînement** | 70% | Apprentissage des modèles |
| **Validation** | 15% | Ajustement des hyperparamètres (section 6) |
| **Test** | 15% | Évaluation finale (jamais vus pendant l'entraînement) |

**Points clés :**
- *Stratification* → préserve le ratio de classes (75/25) dans chaque ensemble
- Suppression de `enrollee_id` → identifiant sans valeur prédictive
- `random_state=42` → reproductibilité des résultats

In [ ]:
# Préparation des données
X = df.drop(['enrollee_id', 'target'], axis=1)
y = df['target']

# Split 70/15/15 stratifié
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Vérification
print(f"Train: {X_train.shape[0]} | Val: {X_val.shape[0]} | Test: {X_test.shape[0]}")

### 5.2 Pipelines et Modèles

Construction de **pipelines** qui enchaînent prétraitement et modèle en une seule étape :

**Prétraitement automatique :**
- *Caractéristiques numériques* → Imputation (médiane) + Standardisation
- *Caractéristiques catégorielles* → Imputation (mode) + Encodage One-Hot

**Modèles de référence :**
| Modèle | Type | Pourquoi |
|--------|------|----------|
| **Régression Logistique** | Linéaire | Simple, interprétable, référence rapide |
| **Forêt Aléatoire** | Ensemble | Capture les interactions non-linéaires |

**Remarque :** `class_weight='balanced'` → gère automatiquement le déséquilibre 75/25

In [ ]:
# Identification des types de features
num_features = X_train.select_dtypes(include=np.number).columns.tolist()
cat_features = X_train.select_dtypes(include='object').columns.tolist()

# Préprocesseur
preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_features),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_features)
])

# Pipelines avec modèles
lr_pipeline = Pipeline([('preprocessor', preprocessor), ('model', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))])
rf_pipeline = Pipeline([('preprocessor', preprocessor), ('model', RandomForestClassifier(class_weight='balanced', random_state=42))])

print(f"Features: {len(num_features)} numériques, {len(cat_features)} catégorielles")

### 5.3 Entraînement et Validation

**Validation croisée 5-fold** sur l'ensemble d'entraînement pour évaluer la performance des modèles de manière robuste.

*Principe :* L'ensemble d'entraînement est divisé en 5 parties → on entraîne sur 4, on valide sur 1, et on répète 5 fois.

**Métriques évaluées :**
- `accuracy` → % de prédictions correctes
- `f1_weighted` → équilibre précision/rappel (adapté aux classes déséquilibrées)
- `roc_auc` → capacité à discriminer les classes

In [ ]:
from sklearn.model_selection import cross_val_score

scoring = ['accuracy', 'f1_weighted', 'roc_auc']

# Cross-validation pour les deux modèles
results = {}
for name, pipeline in [('Logistic Regression', lr_pipeline), ('Random Forest', rf_pipeline)]:
    print(f"\n{name}:")
    results[name] = {}
    for metric in scoring:
        scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring=metric)
        results[name][metric] = scores.mean()
        print(f"  {metric}: {scores.mean():.3f} (+/- {scores.std():.3f})")

### 5.4 Test

### 5.4 Test

**Évaluation finale** sur l'ensemble de test (données jamais vues pendant l'entraînement).

- Entraînement sur `X_train` complet
- Prédiction et évaluation sur `X_test`
- **Rapport de classification** → précision, rappel, F1 par classe
- **Matrice de confusion** → visualisation des erreurs
- **Importance des caractéristiques** → comparaison RF (*Importance de Gini*) vs LR (*coefficients*)

In [ ]:
# Entraînement et évaluation sur test set
models = {'Logistic Regression': lr_pipeline, 'Random Forest': rf_pipeline}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    print(f"\n{'='*50}\n{name}\n{'='*50}")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.3f}")

# Matrice de confusion
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
for idx, (name, model) in enumerate(models.items()):
    cm = confusion_matrix(y_test, model.predict(X_test))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'{name}')
    axes[idx].set_xlabel('Prédit')
    axes[idx].set_ylabel('Réel')
plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance - Comparaison des deux modèles
feature_names = num_features + list(rf_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(cat_features))

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Random Forest - Feature Importances
rf_importances = rf_pipeline.named_steps['model'].feature_importances_
top_rf = np.argsort(rf_importances)[-15:]
axes[0].barh(range(15), rf_importances[top_rf], color='steelblue')
axes[0].set_yticks(range(15))
axes[0].set_yticklabels([feature_names[i] for i in top_rf])
axes[0].set_xlabel('Importance')
axes[0].set_title('Random Forest (Gini Importance)', fontweight='bold')

# Logistic Regression - Coefficients (valeur absolue)
lr_coefs = np.abs(lr_pipeline.named_steps['model'].coef_[0])
top_lr = np.argsort(lr_coefs)[-15:]
axes[1].barh(range(15), lr_coefs[top_lr], color='coral')
axes[1].set_yticks(range(15))
axes[1].set_yticklabels([feature_names[i] for i in top_lr])
axes[1].set_xlabel('|Coefficient|')
axes[1].set_title('Logistic Regression (Coefficient Magnitude)', fontweight='bold')

plt.tight_layout()
plt.show()

### 5.5 Interprétation et Discussion des Résultats

**Comparaison des modèles de référence :**

| Métrique | Régression Logistique | Forêt Aléatoire | Meilleur |
|----------|---------------------|---------------|----------|
| Accuracy | 0,73 | **0,77** | RF |
| ROC-AUC | **0,777** | 0,765 | LR |
| Rappel (classe 1) | **0,70** | 0,39 | LR |
| F1 (classe 1) | **0,57** | 0,45 | LR |

**Conclusions :**

1. **La Régression Logistique surpasse la Forêt Aléatoire** pour l'objectif métier (détecter les candidats cherchant un emploi)
   - Détecte 70% des candidats en recherche d'emploi vs seulement 39% pour RF
   - Meilleur ROC-AUC (0,777 vs 0,765)

2. **L'accuracy est trompeuse** avec des classes déséquilibrées
   - RF a 77% d'accuracy mais rate 61% des candidats à risque

3. **L'importance des caractéristiques révèle des stratégies différentes :**
   - *LR* → exploite fortement la variable `city` (meilleur prédicteur identifié en EDA)
   - *RF* → disperse sur `training_hours` malgré sa faible corrélation linéaire

**Prochaine étape (Section 6) :** Optimiser les hyperparamètres pour améliorer les performances, notamment le rappel de la classe 1.

---

## 6. Modèles Améliorés et Optimisation des Hyperparamètres

### 6.1 Justification des Choix

**Choix du modèle : HistGradientBoostingClassifier**

*Justification basée sur notre analyse :*
- **EDA (Section 4) :** Déséquilibre de classes (75/25), `city` est le meilleur prédicteur (V=0,396), caractéristiques catégorielles à haute cardinalité
- **Références (Section 5) :** Forêt Aléatoire sous-performe sur le rappel classe 1 (0,39), LR meilleure mais limitée aux relations linéaires
- **Cours :** « RandomForest, AdaBoost, GBRT, et HGB sont parmi les premiers modèles à tester »

*Pourquoi HistGradientBoosting :*
- **Gradient Boosting** → souvent meilleur que Forêt Aléatoire sur données tabulaires
- Implémentation **optimisée** dans scikit-learn (rapide)
- Gère bien les **interactions non-linéaires** entre caractéristiques

*Prétraitement :* Même pipeline que les références (imputation + encodage one-hot) pour comparaison équitable.

**Métriques d'optimisation :** `roc_auc` et `f1_weighted` (adaptées au déséquilibre de classes)

### 6.2 Optimisation des Hyperparamètres

**GridSearchCV** pour trouver les meilleurs hyperparamètres :
- `max_depth` → profondeur des arbres (contrôle l'overfitting)
- `learning_rate` → vitesse d'apprentissage
- `max_iter` → nombre d'arbres dans l'ensemble

Validation sur `X_val` (ensemble de validation créé en 5.1).

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Pipeline avec preprocessor (réutilise celui de la section 5.2)
hgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', HistGradientBoostingClassifier(class_weight='balanced', random_state=42))
])

# Grille d'hyperparamètres (préfixe 'model__' pour le pipeline)
param_grid = {
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.05, 0.1, 0.2],
    'model__max_iter': [100, 200]
}

# GridSearchCV avec les deux métriques
results_grid = {}
for scoring in ['roc_auc', 'f1_weighted']:
    print(f"\n{'='*50}\nOptimisation pour: {scoring}\n{'='*50}")
    
    grid_search = GridSearchCV(hgb_pipeline, param_grid, cv=5, scoring=scoring, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    results_grid[scoring] = grid_search
    print(f"Meilleurs paramètres: {grid_search.best_params_}")
    print(f"Meilleur score CV: {grid_search.best_score_:.3f}")

In [ ]:
# Comparaison des deux modèles optimisés (ROC-AUC vs F1)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for idx, (metric, label) in enumerate([('roc_auc', 'ROC-AUC'), ('f1_weighted', 'F1-weighted')]):
    best_model = results_grid[metric].best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)[:, 1]
    
    print(f"\n{'='*50}\nHGB optimisé pour {label}\n{'='*50}")
    print(f"Params: {results_grid[metric].best_params_}")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.3f}")
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'HGB optimisé {label}')
    axes[idx].set_xlabel('Prédit')
    axes[idx].set_ylabel('Réel')

plt.tight_layout()
plt.show()

### 6.3 Résultats Finaux et Analyse

**Comparaison finale de tous les modèles (Ensemble de Test) :**

| Modèle | ROC-AUC | Rappel (classe 1) | F1 (classe 1) | Candidats détectés |
|--------|---------|-------------------|---------------|-------------------|
| Régression Logistique | 0,777 | 0,70 | 0,57 | 504/716 (70%) |
| Forêt Aléatoire | 0,765 | 0,39 | 0,45 | 276/716 (39%) |
| **HGB (ROC-AUC)** | **0,784** | **0,76** | **0,59** | **542/716 (76%)** |
| HGB (F1) | 0,785 | 0,74 | 0,59 | 530/716 (74%) |

**Meilleur modèle : HistGradientBoosting optimisé pour ROC-AUC**

*Hyperparamètres :* `max_depth=5`, `learning_rate=0,05`, `max_iter=100`

**Gains par rapport aux références :**
- **+0,7%** ROC-AUC vs Régression Logistique
- **+6 points** de rappel vs LR → détecte 38 candidats supplémentaires
- **+37 points** de rappel vs Forêt Aléatoire → détecte 266 candidats supplémentaires

**Interprétation métier :** Sur 716 candidats cherchant un emploi, le modèle optimisé en identifie correctement **542 (76%)**, contre 504 pour LR et seulement 276 pour RF. Cette amélioration permet un ciblage plus efficace des candidats à risque d'attrition.

---

## 7. Conclusion

**Objectif :** Prédire les candidats cherchant à changer d'emploi à partir de leurs caractéristiques démographiques, éducatives et professionnelles.

**Principales découvertes (EDA) :**
- Jeu de données de 19 158 candidats avec déséquilibre de classes (75/25)
- `city` et `city_development_index` sont les meilleurs prédicteurs
- Le clustering révèle un segment à haut risque (47% de mobilité)

**Comparaison des modèles :**

| Modèle | ROC-AUC | Rappel (classe 1) |
|--------|---------|-------------------|
| Régression Logistique | 0,777 | 70% |
| Forêt Aléatoire | 0,765 | 39% |
| **HGB (Optimisé)** | **0,784** | **76%** |

**Meilleur modèle :** HistGradientBoostingClassifier
- Hyperparamètres : `max_depth=5`, `learning_rate=0,05`, `max_iter=100`
- Détecte **76% des candidats** cherchant un emploi (542/716)

**Limites :**
- Déséquilibre de classes impacte la précision (49% de faux positifs)
- Caractéristique `city` à haute cardinalité (123 valeurs) → encodage one-hot crée 123 caractéristiques

**Pistes d'amélioration :**
- Encodage cible pour `city` (réduire la dimensionnalité)
- Tester d'autres modèles (XGBoost, LightGBM)
- Ajuster le seuil de décision pour équilibrer précision/rappel